In [12]:
from findmycells.main import Project
import os
import pandas as pd

In [13]:
if os.path.isdir('/mnt/c/Users/dsege/TEMP/test_project2/'):
    project_root_dir = '/mnt/c/Users/dsege/TEMP/test_project2/'
    from findmycells.preprocessing import CropStitchingArtefacts
    from findmycells.segmentation import Deepflash2BinaryAndInstanceSegmentationStrategy
    from findmycells.quantifications import ReconstructCellsIn3DFrom2DInstanceLabels, CountCellsInWholeStructure
    from findmycells.inspection import InspectReconstructedCellsBasedOnMultiMatchIDX
else:
    project_root_dir = '/Users/Sofie/Documents/DCL/Project_root_directory/'

In [14]:
user_input_via_gui = {'project_root_dir': project_root_dir}
os.listdir(project_root_dir)

['02_preprocessed_images',
 '06_inspection',
 '.DS_Store',
 '05_instance_segmentations',
 '04_binary_segmentations',
 '01_rois_to_analyze',
 '07_results',
 '00_microscopy_images',
 '03_deepflash2']

In [15]:
test_project = Project(user_input_via_gui)

In [16]:
# initial check whether also the dictionary was created properly:
test_project.database.file_infos

{'file_id': ['0000', '0001', '0002', '0003'],
 'original_file_id': ['#9_SN_wt_s2_sfr1',
  '#7_SN_wt_s2_sfr1',
  '#8_SN_tg_s2_sfr1',
  '#5_SN_tg_s2_sfr1'],
 'group_id': ['wt', 'wt', 'tg', 'tg'],
 'subject_id': ['#9_wt_old', '#7_wt_old', '#8_tg_old', '#5_tg_old'],
 'microscopy_filepath': ['/Users/Sofie/Documents/DCL/Project_root_directory/00_microscopy_images/wt/#9_wt_old/#9_SN_wt_s2_sfr1.czi',
  '/Users/Sofie/Documents/DCL/Project_root_directory/00_microscopy_images/wt/#7_wt_old/#7_SN_wt_s2_sfr1.czi',
  '/Users/Sofie/Documents/DCL/Project_root_directory/00_microscopy_images/tg/#8_tg_old/#8_SN_tg_s2_sfr1.czi',
  '/Users/Sofie/Documents/DCL/Project_root_directory/00_microscopy_images/tg/#5_tg_old/#5_SN_tg_s2_sfr1.czi'],
 'microscopy_filetype': ['.czi', '.czi', '.czi', '.czi'],
 'rois_present': [True, True, True, True],
 'rois_filepath': ['/Users/Sofie/Documents/DCL/Project_root_directory/01_rois_to_analyze/wt/#9_wt_old/#9_SN_wt_s2_sfr1.roi',
  '/Users/Sofie/Documents/DCL/Project_root_dire

In [19]:
dataframe_files = test_project.database.file_infos #this is what you should convert

In [27]:
#Try to make a dataframe
dataframe_try = pd.DataFrame(dataframe_files)
Database_files_DF = dataframe_try.rename(columns={"file_id": "File ID", "original_file_id": "Original File ID", 
                                           "group_id": "Group ID", "subject_id": "Subject ID", "microscopy_filepath": 
                                          "Microscopy Filepath", "microscopy_filetype": "Microscopy Filetype",
                                          "rois_present": "Rois Present", "rois_filepath": "Rois Filepath",
                                          "rois_filetype": "Rois Filetype"})

Database_files_DF.to_csv("/Users/Sofie/Documents/DCL/DCLmeetsRETUNE/Database_files_DF.csv")

In [6]:
# if yes, let´s continue with loading the results:
test_project.load_status()

ModuleNotFoundError: No module named 'torch'

In [23]:
# now let´s make a copy of the dictionary we want to convert into a pandas DataFrame object
file_infos = test_project.database.file_infos.copy()

# and check whether the results were actually loaded - this should return a list like: [True, True, True, True]
file_infos['preprocessing_completed']

[True, True, True, True]

If this is also the case, it´s now your turn to figure out how to convert the dictionary into a DataFrame and from there to save it as csv file into the results directory (path is stored as attribute in the database). If you get stuck at some point, just sent me an e-mail :)

Hint: you would want to use the [pandas](https://pandas.pydata.org/) package for this. You also need to import the package like this: 

In [ ]:
#If this happens to throw an error, make sure to install pandas to your conda environment 
# using `conda install pandas` in your terminal
import pandas as pd

If you got the first part working, the second part will require you to actually integrate the code you wrote into the "database.py" module so that it becomes part of the pipeline. For this, please create a new method in the Database class that contains all the code to save the file_infos dictionary as csv file into the results directory and a piece of code that calls this method in the "save_all()" method of the Database class (I added a comment at the corresponding position in line 278). 

In order for you to test whether your integrated code works, make sure to save all changes in the 'database.py' file (via clicking "File" in the very upper left corner of your Jupyter-lab browser window - and then "Save Python File") - then restart the Kernel of this notebook (via clicking "Kernel" and "Restart Kernel..."). This is neccessary because you have to import the new version of the database.py file, and that only works after restarting the kernel and re-executing the import. Now create the test_project again, load all results using load_status() and now try running: `test_project.save_all()` Did this save the csv file in the correct directory? :D

### Option A: start a new project

In [3]:
user_input_via_gui = {'project_root_dir': project_root_dir,
                      'preprocessing_configs': {0: {'method_category': None,
                                                    'method_specifier': None,
                                                    'additional_settings': dict(),
                                                    'ProcessingMethod': CropStitchingArtefacts(),
                                                    'ProcessingStrategy': None}
                                               },
                     'segmentation_strategy': Deepflash2BinaryAndInstanceSegmentationStrategy(),
                     'quantification_configs': {'segmentations_to_use': 'instance', # or 'binary'
                                                'quantification_preprocessing_strategies': [ReconstructCellsIn3DFrom2DInstanceLabels()],
                                                'quantification_strategy': CountCellsInWholeStructure(),
                                                'additional_settings': {'min_roi_size': 400,
                                                                        'min_z_extension': 2}},
                     'low_memory': True}

In [4]:
test_project = Project(user_input_via_gui)

### Option b: load status from an existing project:

In [5]:
#test_project = Project({'project_root_dir': project_root_dir})

In [5]:
test_project.load_status()

In [6]:
test_project.database.file_infos['quantification_completed']

[True, True, True, True]

In [9]:
test_project.run_quantificatons()

In [7]:
test_project.database.quantification_results

{'0000': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 81}},
 '0001': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 19}},
 '0002': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 77}},
 '0003': {'000': {'inclusion_criteria_position': ['within', 'intersects'],
   'inclusion_criteria_min_z_extension': 2,
   'number_connected_components': 131}}}

In [14]:
test_project.database.multi_matches_traceback['0000']['original_instance_label_id'][22]

26

In [ ]:
test_project.run_inspection('0000', InspectReconstructedCellsBasedOnMultiMatchIDX(22, save=True, show=True))

In [16]:
test_project.save_status()

# This is where we want to go:

In [ ]:
## Initialize the project

project_name = Project(project_root_dir)
# project_root_dir: path of project root directory

project_name.initialize_database()
# creates the database object and sets-up everything (main subdirs, file infos, ..)
# as alternative, see: load_status()

project_name.save_current_status()
# saves current progress status to disk (basically the database.file_infos as pickle file? - what about shapely polygons that might have been loaded?)


project_name.load_status()
# Alternative to initialize_database()
# loads the corresponding information from disk and allows the user to continue from here


## Preprocessing

project_name.preprocess(microscopy_files, cropping_strategy)
# microscopy_files: list of all microscopy image files that shall be preprocessed (this would allow step-by-step processing)
# cropping_strategy: object of type CroppingStrategy that specifies which exact cropping strategy shall be used

# infers:
    # microscopy-image-file-loader from filetype (e.g. .czi file loader)
    # whether it´s a z-stack
    # whether it´s an RGB image
# automatically updates the database accordingly


## Image segmentation

project_name.run_segmentations(files, SegmentationStrategy)
# files: list of files that shall be segmented (in case of df2 make sure that all are used due to image statistics & cellpose diameter)
# SegmentationStrategy: e.g. deepflash2_binary, deepflash2_instances, intellesis

# launches the respective segmentations and saves all progress in the database object & all segmentations in the corresponding subdir


## Quantifications

project_name.run_quantifications(files, QuantificationStrategy)
# files: list of files whose segmentations shall be used for quantification
# QuantificationStrategy: e.g. 2D, 3D_grid_estimation, 3D_entire_volume (enable selection of more specific options / exclusion criteria)

# Launches the respective quantifications and saves all progress in the database object & all quantification results in the results table